In [1]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

!pip install ipython-autotime
%load_ext autotime

time: 1.54 ms (started: 2022-03-24 21:08:12 +00:00)


### Download & Process Glove. Ignore if mod file exists

In [2]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip

model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt')
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

generate_norm_embedding(model_glove,'glove_norm_300.mod')


--2022-03-24 21:08:52--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-03-24 21:08:52--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  5.19MB/s    in 5m 54s  

2022-03-24 21:14:46 (5.06 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 13min 40s (started: 2022-03-24 21:08:52 +00:00)


### Load processed glove

In [ ]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss


time: 33.5 s (started: 2022-03-23 19:02:57 +00:00)


In [3]:
current_model = model_gn


time: 4.41 ms (started: 2022-03-24 21:27:52 +00:00)


### Load and process antonyms. Ignore if final_antonym_list exists.

In [ ]:
list_antonym = []

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))

with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

### Load final_antonym_list as antonym list

In [4]:
import pandas as pd

list_antonym = pd.read_pickle(r'final_antonym_list')

time: 1.02 s (started: 2022-03-24 21:28:33 +00:00)


### Decide on the size of the antonym vector


In [5]:
num_antonym = 1468

## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 45 ms (started: 2022-03-24 21:28:44 +00:00)


### Subset Dimension Selection Method


In [6]:
import random
import scipy

random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = scipy.spatial.distance.cdist(np.array(antonymy_vector),np.array(antonymy_vector),'cosine')
dimension_similarity_matrix = abs(1-dimension_similarity_matrix)
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
    return final_position_index

time: 631 ms (started: 2022-03-24 21:28:49 +00:00)


## Select POLAR dimensions

#### Generate the RANDOM DIMENSION Order


In [7]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

1468
time: 4.61 ms (started: 2022-03-24 21:28:54 +00:00)


#### Generate the ORTHOGONAL DIMENSION Order


In [8]:
orthogonal_antonymy_vector =np.array(select_subset_dimension(antonymy_vector, num_antonym))  
print(orthogonal_antonymy_vector.shape)

working list is ready, shape (1468, 300)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1467 [00:00<?, ?it/s]

(1468,)
time: 3min 12s (started: 2022-03-24 21:28:56 +00:00)


#### Generate the MAXIMUM VARIANCE DIMENSION Order


In [9]:
import torch

embedding_size = antonymy_vector.shape[0]   
print('The embedding size is', embedding_size)


variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vector))
variance_antonymy_vector_inverse = torch.tensor(variance_antonymy_vector_inverse)

embedding_matrix = []

current_model_tensor = torch.t(torch.tensor(current_model.wv.vectors))

The embedding size is 1468


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


time: 10.4 s (started: 2022-03-24 21:33:08 +00:00)


In [18]:
var_list = [None for x in range(20)] # variance for each antonym in each batch

for i in range(19):  # the first 19 batches, each of size 100k
  temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,100000*i:100000*i+100000])
  temp_var_mean = torch.var(temp, axis = 1)
  var_list[i] = temp_var_mean.numpy()
  del temp

temp = torch.matmul(variance_antonymy_vector_inverse, current_model_tensor[:,1900000:])
temp_var_mean = torch.var(temp, axis = 1)
var_list[19] = temp_var_mean.numpy()
del temp

time: 50.6 s (started: 2022-03-24 21:35:34 +00:00)


In [19]:
# using lazy approach. assume each batch is independent and the overall variance is the average variance over all batches

variance_list = np.mean(np.array(var_list),axis = 0)

variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]


time: 6.19 ms (started: 2022-03-24 21:36:31 +00:00)


### Begin POLAR embedding

In [21]:
import pandas as pd

company = pd.read_csv('glove-ticker-fortune1000-us.csv')
ticker_list = company['Ticker'].str.lower() # a list of tickers

time: 56.5 ms (started: 2022-03-24 21:36:43 +00:00)


In [22]:
ticker_word_embedding = dict()
for ticker in ticker_list:    # dictionary of ticker: glove embedding
  ticker_word_embedding[ticker] = current_model[ticker]

time: 31 ms (started: 2022-03-24 21:36:51 +00:00)


In [23]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 31.7 ms (started: 2022-03-24 21:36:55 +00:00)


In [25]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 13.5 ms (started: 2022-03-24 21:38:30 +00:00)


Select the number of POLAR demensions and the method of antonym selection

In [35]:
import subprocess
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = orthogonal_antonymy_vector 
# random_antonym_vector or orthogonal_antonymy_vector or variance_antonymy_vector 
antonym_500 = [list_antonym[x] for x in antonym_vector_method[:500]]


time: 3.41 ms (started: 2022-03-24 21:41:50 +00:00)


In [36]:
# create POLAR embedding of tickers
ticker_new_embedding = generate_embedding_path(ticker_word_embedding, 'test_run',True ,antonym_vector_method,500) 


New model size is 892 500


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/892 [00:00<?, ?it/s]

time: 629 ms (started: 2022-03-24 21:41:57 +00:00)


#### View top embeddings

In [28]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

time: 9.99 ms (started: 2022-03-24 21:39:25 +00:00)


In [37]:
# amazon 
amzn_polar = make_polar_dict('amzn', antonym_500, ticker_new_embedding, True)


('ceo', 'workforce') 0.10772923 

('punch', 'stroke') 0.10793858 

('realism', 'symbolism') -0.1083337 

('condenser', 'stretcher') 0.11130721 

('test', 'work') 0.1145705 

('outside', 'room') 0.12066327 

('accidental', 'intentional') -0.12215772 

('maximum', 'minimum') -0.12317282 

('taxonomic', 'unclassified') -0.14370447 

('apple', 'tomato') 0.15129773 

time: 6 ms (started: 2022-03-24 21:42:04 +00:00)


In [38]:
# walmart
wmt_polar = make_polar_dict('wmt', antonym_500, ticker_new_embedding, True)


('desert', 'saloon') -0.12026095 

('saunter', 'whisk') 0.12478568 

('saloon', 'teahouse') -0.1254237 

('apple', 'tomato') 0.12660202 

('maximum', 'minimum') -0.12834348 

('fortune', 'trouble') 0.13408856 

('critic', 'musician') 0.13633755 

('outside', 'room') 0.13712038 

('airless', 'pneumatic') -0.14000669 

('alphabet', 'count') -0.1476502 

time: 3.56 ms (started: 2022-03-24 21:42:15 +00:00)


In [39]:
# facebook
fb_polar = make_polar_dict('fb', antonym_500, ticker_new_embedding, True)


('singular', 'tribal') -0.107639536 

('curve', 'square') 0.10996485 

('lone', 'tribal') 0.110461906 

('extrasensory', 'sensory') 0.11196343 

('drama', 'parody') 0.11261892 

('ceo', 'workforce') 0.11463114 

('unemployed', 'workforce') -0.115528435 

('group', 'separate') 0.11801642 

('captain', 'follower') -0.1252387 

('follower', 'innovator') 0.14101696 

time: 3.34 ms (started: 2022-03-24 21:42:18 +00:00)


In [40]:
# wells fargo
wfc_polar = make_polar_dict('wfc', antonym_500, ticker_new_embedding, True) 


('accidental', 'intentional') -0.10983897 

('outside', 'room') 0.113385014 

('bat', 'field') 0.1184059 

('hour', 'second') -0.11888394 

('hardware', 'software') 0.11955834 

('confirm', 'question') 0.12253661 

('hail', 'snow') -0.12533107 

('cynicism', 'naivety') -0.12634936 

('credit', 'tax') 0.12675561 

('ceo', 'workforce') 0.14842771 

time: 3.62 ms (started: 2022-03-24 21:42:21 +00:00)
